# 캘리포니아 주택 가격 예측

### 기본 정보

- 인구, 중간소득 등의 특성을 사용하여 주택 가격 예측
- 다중 회귀, 단방향 회귀(반대 : 다변량 회귀)
    - 다중 회귀 : 여러 개의 특성을 이용한 회귀
    - 단방향 회귀 : 하나의 데이터에 대한 예측
    - 다변량 회귀 : 여러 개의 수치를 예측하기 위한 회귀
- 회귀의 성능 측정 방법
    - 평균 제곱근의 오차를 활용
    - 평균 절대오차를 활용

- 일차적으로 파이썬 커널을 설정하면서 정상적으로 import가 되는지 체크해주었다. 
- 커널에 원하는 import 추가하는 법
    1. `venv`로 커널을 생성해준다. 
    2. `source .venv/bin/activate`로 venv를 활성화해준다. 
    3. 내부에서 `pip install`로 원하는 라이브러리를 다운로드 받는다. 

In [4]:
import sys

assert sys.version_info >= (3, 5)

import sklearn

assert sklearn.__version__ >= "0.20"

import numpy as np
import os

import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

ROOT_DIR = '.'
CHAPTER_ID = 'end_to_end_project'
IMAGE_PATH = os.path.join(ROOT_DIR, 'images', CHAPTER_ID)
os.makedirs(IMAGE_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension = 'png', resolution = 100): 
    path = os.path.join(IMAGE_PATH, fig_id + '.' + fig_extension)
    print('그림 저장 : ', fig_id)

    if tight_layout:
        plt.tight_layout()
    
    plt.savefig(path, format=fig_extension, dpi=resolution)

import warnings

warnings.filterwarnings(action='ignore', message='^internal gelsd')


Matplotlib is building the font cache; this may take a moment.
